In [1]:
from interactions import get_interactions
import numpy as np
import os


In [2]:
X = 128
Y = 128
Y2 = int(Y/2)

Get interactions for first sublattice for a specific disorder sample

In [3]:
# Read data from interactions file
with open('/home/dfki.uni-bremen.de/lpwichette/Desktop/Repos/qec/optim/results/HamiltonianTest/hamiltxspin/interaction_e00.txt', 'r') as file:
    content = file.read()

# Split the content into lines and then into individual numbers
lines = content.strip().split('\n')
numbers = [list(map(int, line.split())) for line in lines]

# Convert the list of lists into a numpy array
array_interactions = np.array(numbers)

# Now, 'array' contains your data in a numpy array format
np.set_printoptions(threshold=np.inf)
array_interactions.shape

interactions = get_interactions(array_interactions, X, X)

In [4]:
old_interactions = interactions
new_interactions = array_interactions


Get black and white first sublattice of first update step for current error sample 

In [5]:
# Read the content of the text file
with open('/home/dfki.uni-bremen.de/lpwichette/Desktop/Repos/qec/optim/results/HamiltonianTest/hamiltxspin/lattice_e0_j0.txt', 'r') as file:
    content = file.readlines()

# Remove any newline characters
content = [line.strip() for line in content]

# Convert the content into a numpy array of integers
lattice = np.array([[int(char) for char in line] for line in content])

# Get only first sublattice
lattice = lattice[:,:]

In [6]:
lattice[lattice==0] = -1


In [7]:
lattice

array([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        -1, -1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,

In [8]:
lattice_b = np.zeros((X,Y2))
lattice_w = np.zeros((X,Y2))
# Get black and white lattices
for i in range(X):
    if i%2 == 0:
        lattice_b[i,:] = lattice[i,0::2]
        lattice_w[i,:] = lattice[i,1::2]
    else:
        lattice_b[i,:] = lattice[i,1::2]
        lattice_w[i,:] = lattice[i,0::2]

In [9]:
# # Check whether
# is_black = False

In [10]:
hamiltonian = 0
for is_black in (True, False):
    for tid in range(X*Y2):

        i = int(tid/Y2)
        j = tid % Y2

        ipp = (i + 1) if (i + 1 < X) else 0
        inn = (i - 1) if (i - 1 >= 0) else X - 1
        jpp = (j + 1) if (j + 1 < Y2) else 0
        jnn = (j - 1) if (j - 1 >= 0) else Y2 - 1

        c_up = 1-int(inn/(X-1))
        c_down = 1-int((i+1)/X)

        if (is_black):

            icpp = 2*(X-1)*Y2 + 2*(Y2*(i+1) + j) + i%2

            icnn = 2*(X-1)*Y2 + 2*(Y2*(inn+1) + j) + i%2

            joff = jpp if (i%2) else jnn

            if (i % 2):
                c_side = 1-int((j+1)/Y2)

                if (j+1 >= Y2):
                    jcoff = 2*(i*Y2 + j + 1) - 1

                else:
                    jcoff = 2*(i*Y2 + joff) - 1
            else:
                c_side = 1-int(jnn/(Y2-1))

                jcoff = 2 * (i*Y2 + joff) + 1

            hamiltonian += lattice_b[i,j]*(c_up*lattice_w[inn,j]*interactions[icnn]+c_down*lattice_w[ipp,j]*interactions[icpp]+1*lattice_w[i,j]*interactions[2*(i*Y2+j)]+c_side*lattice_w[i, joff]*interactions[jcoff])

        else:

            icpp = 2*(X - 1)*Y2 + 2*(Y2*(i+1) + j) + (i+1)%2
            icnn = 2*(X - 1)*Y2 + 2*(Y2*(inn+1) + j) + (i+1)%2

            joff = jnn if (i%2) else jpp

            if (i%2):
                c_side = 1-int(jnn/(Y2-1))

                jcoff = 2*(i*Y2 + joff) + 1
            else:

                c_side = 1-int((j+1)/Y2)


                if (j+1 >= Y2):
                    jcoff = 2*(i*Y2 + j + 1) - 1
                else:
                    jcoff = 2*(i*Y2 + joff) - 1

            hamiltonian += lattice_w[i,j]*(c_up*lattice_b[inn,j]*interactions[icnn]+c_down*lattice_b[ipp,j]*interactions[icpp]+1*lattice_b[i,j]*interactions[2*(i*Y2+j)]+c_side*lattice_b[i, joff]*interactions[jcoff])
hamiltonian


51544.0

In [11]:
hamiltonian = np.zeros((X,Y))
for is_black in (True, False):
    for tid in range(X*Y2):

        i = int(tid/Y2)
        j = tid % Y2

        ipp = (i + 1) if (i + 1 < X) else 0
        inn = (i - 1) if (i - 1 >= 0) else X - 1
        jpp = (j + 1) if (j + 1 < Y2) else 0
        jnn = (j - 1) if (j - 1 >= 0) else Y2 - 1

        c_up = 1-int(inn/(X-1))
        c_down = 1-int((i+1)/X)

        if (is_black):

            icpp = 2*(X-1)*Y2 + 2*(Y2*(i+1) + j) + i%2

            icnn = 2*(X-1)*Y2 + 2*(Y2*(inn+1) + j) + i%2

            joff = jpp if (i%2) else jnn

            if (i % 2):
                c_side = 1-int((j+1)/Y2)

                if (j+1 >= Y2):
                    jcoff = 2*(i*Y2 + j + 1) - 1

                else:
                    jcoff = 2*(i*Y2 + joff) - 1
            else:
                c_side = 1-int(jnn/(Y2-1))

                jcoff = 2 * (i*Y2 + joff) + 1

            if (i%2==0):
                hamiltonian[i][2*j]=lattice_b[i,j]*(c_up*lattice_w[inn,j]*interactions[icnn]+c_down*lattice_w[ipp,j]*interactions[icpp]+1*lattice_w[i,j]*interactions[2*(i*Y2+j)]+c_side*lattice_w[i, joff]*interactions[jcoff])
            else:
                hamiltonian[i][2*j+1]=lattice_b[i,j]*(c_up*lattice_w[inn,j]*interactions[icnn]+c_down*lattice_w[ipp,j]*interactions[icpp]+1*lattice_w[i,j]*interactions[2*(i*Y2+j)]+c_side*lattice_w[i, joff]*interactions[jcoff])

        else:

            icpp = 2*(X - 1)*Y2 + 2*(Y2*(i+1) + j) + (i+1)%2
            icnn = 2*(X - 1)*Y2 + 2*(Y2*(inn+1) + j) + (i+1)%2

            joff = jnn if (i%2) else jpp

            if (i%2):
                c_side = 1-int(jnn/(Y2-1))

                jcoff = 2*(i*Y2 + joff) + 1
            else:

                c_side = 1-int((j+1)/Y2)


                if (j+1 >= Y2):
                    jcoff = 2*(i*Y2 + j + 1) - 1
                else:
                    jcoff = 2*(i*Y2 + joff) - 1

            if (i%2==0):
                hamiltonian[i][2*j+1]=lattice_w[i,j]*(c_up*lattice_b[inn,j]*interactions[icnn]+c_down*lattice_b[ipp,j]*interactions[icpp]+1*lattice_b[i,j]*interactions[2*(i*Y2+j)]+c_side*lattice_b[i, joff]*interactions[jcoff])
            else:
                hamiltonian[i][2*j]=lattice_w[i,j]*(c_up*lattice_b[inn,j]*interactions[icnn]+c_down*lattice_b[ipp,j]*interactions[icpp]+1*lattice_b[i,j]*interactions[2*(i*Y2+j)]+c_side*lattice_b[i, joff]*interactions[jcoff])


In [12]:
hamiltonian

array([[ 2.,  3.,  3.,  1.,  1.,  1.,  3.,  3.,  3.,  3., -1.,  1.,  3.,
         3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  1.,
         3.,  3.,  1.,  3.,  3.,  3.,  1.,  3.,  3.,  3.,  3.,  3.,  1.,
         1.,  1.,  3.,  1.,  1.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
         3.,  1.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  1.,  1.,
         1.,  3.,  3.,  3.,  3.,  3., -1.,  1.,  1.,  1.,  3.,  3.,  3.,
         3.,  1.,  1.,  1.,  1.,  3.,  3.,  3.,  3.,  3.,  1.,  1.,  1.,
         3.,  1.,  3.,  3.,  3.,  3.,  1.,  3.,  1.,  1.,  3.,  3.,  3.,
         1.,  1.,  3.,  1.,  1.,  3.,  3.,  3.,  3.,  3.,  1., -1.,  3.,
         1.,  1.,  3.,  3.,  3.,  3.,  3.,  3.,  1.,  1.,  2.],
       [ 3.,  4.,  4.,  4.,  4.,  2.,  2.,  2.,  2.,  2.,  2.,  4.,  4.,
         4.,  4.,  4.,  4.,  2.,  2.,  2.,  4.,  4.,  4.,  4.,  4.,  2.,
         4.,  2.,  2.,  4.,  2.,  4.,  2.,  2.,  4.,  4.,  4.,  4.,  2.,
         4.,  2.,  2.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4

In [15]:
# Open the file in read mode
with open('/home/dfki.uni-bremen.de/lpwichette/Desktop/Repos/qec/optim/results/HamiltonianTest/hamiltxspin/hamilt_x_spin_e0_j0.txt', 'r') as file:
    # Read the entire content of the file
    content = file.read()

# Split the content into individual numbers based on spaces
numbers = list(map(int, content.split()))

# Print the numbers
optimHamilt = np.reshape(numbers, (128,128))

In [16]:
np.shape(np.nonzero(optimHamilt-hamiltonian))
x_nonzero = np.nonzero(optimHamilt-hamiltonian)[0][:]
y_nonzero = np.nonzero(optimHamilt-hamiltonian)[1][:]


In [17]:
x_nonzero[:5]

array([], dtype=int64)

In [18]:
y_nonzero[:5]

array([], dtype=int64)

In [19]:
reshaped_interactions = np.reshape(interactions, (256, 128))

In [20]:
right = reshaped_interactions[0][54]
left = reshaped_interactions[0][53]
down = reshaped_interactions[128][54]

In [21]:
lattice[0][54]

1

In [ ]:
rightneighbor = lattice[0][55]
leftneighbor = lattice[0][53]
downneighbor = lattice[1][54]

In [ ]:
down

-1.0